# Tutorial 07: understanding restrictions and BlockDofMap

In [ ]:
import matplotlib
import matplotlib.collections
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from mpl_toolkits.axes_grid1 import make_axes_locatable
from numpy import array, array2string, unique, where, zeros
from ufl import *
from dolfinx import *
from dolfinx.cpp.generation import UnitDiscMesh
from dolfinx.cpp.mesh import GhostMode, cell_num_entities
from dolfinx.fem import locate_dofs_topological
from dolfinx.io import XDMFFile
from multiphenics import *

### Auxiliary functions for plotting

In [ ]:
def plot_mesh(mesh, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_aspect("equal")
    points = mesh.geometry.points
    tria = tri.Triangulation(points[:, 0], points[:, 1], mesh.cells())
    ax.triplot(tria, color="k")
    return ax

In [ ]:
def plot_mesh_function(mesh_function, ax=None):
    if ax is None:
        ax = plt.gca()
    ax.set_aspect("equal")
    mesh = mesh_function.mesh()
    points = mesh.geometry.points
    cells = mesh.cells()
    colors = ["b", "r"]
    cmap = matplotlib.colors.ListedColormap(colors)
    cmap_bounds = [0, 0.5, 1]
    norm = matplotlib.colors.BoundaryNorm(cmap_bounds, cmap.N)
    assert mesh_function.dim in (mesh.topology.dim, mesh.topology.dim - 1)
    if mesh_function.dim == mesh.topology.dim:
        tria = tri.Triangulation(points[:, 0], points[:, 1], mesh.cells())
        cell_colors = zeros((mesh.cells().shape[0], ))
        for v in (0, 1):
            cell_colors[where(mesh_function.values == v)[0]] = v
        mappable = ax.tripcolor(tria, cell_colors, edgecolor="k", cmap=cmap, norm=norm)
    elif mesh_function.dim == mesh.topology.dim - 1:
        tdim = mesh.topology.dim
        connectivity_cells_to_facets = mesh.topology.connectivity(tdim, tdim - 1)
        connectivity_facets_to_vertices = mesh.topology.connectivity(tdim - 1, 0)
        linestyles = [(0, (5, 10)), "solid"]
        lines = list()
        lines_colors_as_int = list()
        lines_colors_as_str = list()
        lines_linestyles = list()
        for c in range(mesh.num_cells()):
            facets = connectivity_cells_to_facets.links(c)
            for f in facets:
                mesh_function_f = mesh_function.values[f]
                vertices = connectivity_facets_to_vertices.links(f)
                lines.append(points[vertices][:, :2])
                lines_colors_as_int.append(mesh_function_f)
                lines_colors_as_str.append(colors[mesh_function_f])
                lines_linestyles.append(linestyles[mesh_function_f])
        mappable = matplotlib.collections.LineCollection(lines, cmap=cmap, norm=norm, colors=lines_colors_as_str, linestyles=lines_linestyles)
        mappable.set_array(array(lines_colors_as_int))
        ax.add_collection(mappable)
        ax.autoscale()
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(mappable, cax=cax, cmap=cmap, norm=norm, boundaries=cmap_bounds, ticks=cmap_bounds)
    return ax

In [ ]:
def plot_dofmap(V, ax=None):
    if ax is None:
        ax = plt.gca()
    text_offset = [1e-2, 1e-2]
    coordinates = V.tabulate_dof_coordinates().round(decimals=3)
    ax.scatter(coordinates[:, 0], coordinates[:, 1], c="k", s=50)
    for c in unique(coordinates, axis=0):
        dofs_c = (coordinates == c).all(axis=1).nonzero()[0]
        text_c = array2string(dofs_c, separator=", ", max_line_width=10)
        ax.text(c[0] + text_offset[0], c[1] + text_offset[1], text_c, fontsize=20)
    return ax

### Mesh

Read in a simple mesh consisting in an hexagon discretized with six equilateral triangle cells.

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon.xdmf") as infile:
    mesh = infile.read_mesh(GhostMode.none)

In [ ]:
plot_mesh(mesh)

### Mesh restrictions on cells

Read in a mesh function on cells, which is equal to one on all cells.

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_cell_restriction_all.xdmf") as infile:
    cell_restriction_all = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(cell_restriction_all)

Read in a mesh function on cells, which is equal to zero on one half of the cells, and equal to one on the other half of the cells

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_cell_restriction_subset.xdmf") as infile:
    cell_restriction_subset = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(cell_restriction_subset)

### Mesh restrictions on facets

Define a mesh function on facets, which is equal to one on all facets

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_facet_restriction_all.xdmf") as infile:
    facet_restriction_all = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(facet_restriction_all)

In [ ]:
with XDMFFile(MPI.comm_world, "data/hexagon_facet_restriction_subset.xdmf") as infile:
    facet_restriction_subset = infile.read_mf_size_t(mesh)

In [ ]:
plot_mesh_function(facet_restriction_subset)

In [ ]:
cell_restrictions = (cell_restriction_all, cell_restriction_subset)
facet_restrictions = (facet_restriction_all, facet_restriction_subset)
all_restrictions = cell_restrictions + facet_restrictions

### Lagrange spaces

Define Lagrange FE spaces of order $k=1, 2, 3$, and plot the associated DofMap.

In [ ]:
CG = [FunctionSpace(mesh, ("Lagrange", k)) for k in (1, 2, 3)]

In [ ]:
_, ax = plt.subplots(3, 1, figsize=(10, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k])
    plot_dofmap(CGk, ax[k])
    ax[k].set_title("CG " + str(k + 1) + " FunctionSpace", fontsize=30)

Define multiphenics block function spaces associated to the Lagrange FE spaces, for all four restrictions

In [ ]:
block_CG = dict()
for restriction in all_restrictions:
    block_CG[restriction] = list()
    for CGk in CG:
        restrict_CGk = locate_dofs_topological(CGk, restriction.dim, where(restriction.values == 1)[0])
        block_CG[restriction].append(BlockFunctionSpace([CGk], restrict=[restrict_CGk]))

Compare DOFs for the case of cell restriction equal to one on the entire domain. There is indeed no difference.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CGk, ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " FunctionSpace", fontsize=30)
for (k, block_CGk) in enumerate(block_CG[cell_restriction_all]):
    plot_mesh_function(cell_restriction_all, ax[k, 1])
    plot_dofmap(block_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " BlockFunctionSpace", fontsize=30)

Compare DOFs for che case of cell restriction equal to one on a subset of the domain. Note how the BlockFunctionSpace has only a subset of the DOFs of the FunctionSpace, and properly renumbers them.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CGk, ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " FunctionSpace", fontsize=30)
for (k, block_CGk) in enumerate(block_CG[cell_restriction_subset]):
    plot_mesh_function(cell_restriction_subset, ax[k, 1])
    plot_dofmap(block_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " BlockFunctionSpace", fontsize=30)

Compare DOFs for che case of facet restriction equal to one on the entire domain. Note how there is no difference for $k=1, 2$, but the cases $k=3$ differ (the BlockFunctionSpace does not have the DOF at the cell center).

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CG[k], ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " FunctionSpace", fontsize=30)
for (k, block_CGk) in enumerate(block_CG[facet_restriction_all]):
    plot_mesh_function(facet_restriction_all, ax[k, 1])
    plot_dofmap(block_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " BlockFunctionSpace", fontsize=30)

Compare DOFs for che case of facet restriction equal to one on a subset of the domain. Note how the BlockFunctionSpace has only a subset of the DOFs of the FunctionSpace, and properly renumbers them.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, CGk) in enumerate(CG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(CG[k], ax[k, 0])
    ax[k, 0].set_title("CG " + str(k + 1) + " FunctionSpace", fontsize=30)
for (k, block_CGk) in enumerate(block_CG[facet_restriction_subset]):
    plot_mesh_function(facet_restriction_subset, ax[k, 1])
    plot_dofmap(block_CGk, ax[k, 1])
    ax[k, 1].set_title("CG " + str(k + 1) + " BlockFunctionSpace", fontsize=30)

### Discontinuous Galerkin spaces

Define Discontinuous Galerkin spaces of order $k = 0, 1, 2$, and plot the associated DofMap. This spaces will be used in combination with a cell restriction, as DG DOFs are associated to cells.

In [ ]:
DG = [FunctionSpace(mesh, ("Discontinuous Lagrange", k)) for k in (0, 1, 2)]

In [ ]:
_, ax = plt.subplots(3, 1, figsize=(10, 30))
for (k, DGk) in enumerate(DG):
    plot_mesh(mesh, ax[k])
    plot_dofmap(DGk, ax[k])
    ax[k].set_title("DG " + str(k) + " FunctionSpace", fontsize=30)

Define Discontinuous Galerkin Trace spaces of order $k = 0, 1, 2, 3$, and plot the associated DofMap. This spaces will be used in combination with a facet restriction, as DGT DOFs are associated to facets.

In [ ]:
DGT = [FunctionSpace(mesh, ("Discontinuous Lagrange Trace", k)) for k in (0, 1, 2)]

In [ ]:
_, ax = plt.subplots(3, 1, figsize=(10, 30))
for (k, DGTk) in enumerate(DGT):
    plot_mesh(mesh, ax[k])
    plot_dofmap(DGTk, ax[k])
    ax[k].set_title("DGT " + str(k) + " FunctionSpace\n", fontsize=30)

Define multiphenics block function spaces associated to the Discontinuos Galerkin FE spaces, for all cell restrictions

In [ ]:
block_DG = dict()
for restriction in cell_restrictions:
    block_DG[restriction] = list()
    for DGk in DG:
        restrict_DGk = locate_dofs_topological(DGk, restriction.dim, where(restriction.values == 1)[0])
        block_DG[restriction].append(BlockFunctionSpace([DGk], restrict=[restrict_DGk]))

Define multiphenics block function spaces associated to the Discontinuos Galerkin Trace FE spaces, for all facet restrictions

In [ ]:
block_DGT = dict()
for restriction in facet_restrictions:
    block_DGT[restriction] = list()
    for DGTk in DGT:
        restrict_DGTk = locate_dofs_topological(DGTk, restriction.dim, where(restriction.values == 1)[0])
        block_DGT[restriction].append(BlockFunctionSpace([DGTk], restrict=[restrict_DGTk]))

Compare DOFs for the case of cell restriction equal to one on the entire domain. There is indeed no difference.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGk) in enumerate(DG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGk, ax[k, 0])
    ax[k, 0].set_title("DG " + str(k) + " FunctionSpace", fontsize=30)
for (k, block_DGk) in enumerate(block_DG[cell_restriction_all]):
    plot_mesh_function(cell_restriction_all, ax[k, 1])
    plot_dofmap(block_DGk, ax[k, 1])
    ax[k, 1].set_title("DG " + str(k) + " BlockFunctionSpace", fontsize=30)

Compare DOFs for che case of cell restriction equal to one on a subset of the domain. Note how the BlockFunctionSpace has only a subset of the DOFs of the FunctionSpace, and properly renumbers them. Note also that the number of DOFs at the same physical location might be different between FunctionSpace and BlockFunction (see e.g. the center of the hexagon).

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGk) in enumerate(DG):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGk, ax[k, 0])
    ax[k, 0].set_title("DG " + str(k) + " FunctionSpace", fontsize=30)
for (k, block_DGk) in enumerate(block_DG[cell_restriction_subset]):
    plot_mesh_function(cell_restriction_subset, ax[k, 1])
    plot_dofmap(block_DGk, ax[k, 1])
    ax[k, 1].set_title("DG " + str(k) + " BlockFunctionSpace", fontsize=30)

Compare DOFs for che case of facet restriction equal to one on the entire domain. There is indeed no difference.

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGTk) in enumerate(DGT):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGTk, ax[k, 0])
    ax[k, 0].set_title("DGT " + str(k) + " FunctionSpace\n", fontsize=30)
for (k, block_DGTk) in enumerate(block_DGT[facet_restriction_all]):
    plot_mesh_function(facet_restriction_all, ax[k, 1])
    plot_dofmap(block_DGTk, ax[k, 1])
    ax[k, 1].set_title("DGT " + str(k) + " BlockFunctionSpace\n", fontsize=30)

Compare DOFs for che case of facet restriction equal to one on a subset of the domain. Note how the BlockFunctionSpace has only a subset of the DOFs of the FunctionSpace, and properly renumbers them. Note also that the number of DOFs at the same physical location might be different between FunctionSpace and BlockFunction (see e.g. the center of the hexagon).

In [ ]:
_, ax = plt.subplots(3, 2, figsize=(20, 30))
for (k, DGTk) in enumerate(DGT):
    plot_mesh(mesh, ax[k, 0])
    plot_dofmap(DGTk, ax[k, 0])
    ax[k, 0].set_title("DGT " + str(k) + " FunctionSpace\n", fontsize=30)
for (k, block_DGTk) in enumerate(block_DGT[facet_restriction_subset]):
    plot_mesh_function(facet_restriction_subset, ax[k, 1])
    plot_dofmap(block_DGTk, ax[k, 1])
    ax[k, 1].set_title("DGT " + str(k) + " BlockFunctionSpace\n", fontsize=30)